In [1]:
# Take a look at the csv format
import csv
middle_cates = csv.reader(open("middle_categorization.csv"), delimiter=",")
i = 1
for row in middle_cates:
    print(row)
    if i>1: print(int(row[0])==8066)
    if i==3: break
    i += 1

['race id', '', 'required skills', '', 'Note', '', 'answer', 'BERT_Base']
['8066', '1', '0', '', '', '', '3', '3']
True
['8066', '2', '0', '8', 'early in the morning', '', '0', '0']
True


In [2]:
# Take a look at the prediction file
def read_pred(file):
    for line in file:
        yield line
    
file = open("BERT_Base/analyze_middle.txt", "r")
middle_pred_ite = read_pred(file)
for i in range(17):
    row = next(middle_pred_ite)
    print("THIS IS ROW %s" % str(i+1))
    print(row.strip()=='race_id:8066')
    print(row)

THIS IS ROW 1
True
race_id:8066

THIS IS ROW 2
False


THIS IS ROW 3
False
answer: 3

THIS IS ROW 4
False
prediction: 3

THIS IS ROW 5
False


THIS IS ROW 6
False
choice: 0

THIS IS ROW 7
False
[CLS] children in england must ##n ' t work until they are 13 . they need to have a work permit to start working . the jobs teenagers can do delivering newspapers many teenagers will get up early to deliver newspapers to houses in their local area before going to school . they are known as paper - boys or paper ##girl ##s . baby ##sit ##ting : looking after young children in their home while their parents have gone out for the evening is a popular job for teenagers , as they get money for watching children and television all at the same time ! helping the milk ##man : from the age of 14 some teenagers help the milk ##man deliver milk to houses . other popular jobs : working in a shop ; office work ; washing cars ; in a cafe or restaurant . the hours teenagers ( 13 and 14 year olds ) can work : s

In [3]:
# Take a look at the prediction file
def read_pred(file):
    for line in file:
        yield line
    
file = open("BERT_Base/analyze_middle.txt", "r")
middle_pred_ite = read_pred(file)
pqa = []
for i in range(17):
    pqa.append(next(middle_pred_ite).strip())
for i in range(17):
    print("THIS IS ROW %s" % str(i+1))
    print(pqa[i])
print(pqa[0][8:] == "8066")

THIS IS ROW 1
race_id:8066
THIS IS ROW 2

THIS IS ROW 3
answer: 3
THIS IS ROW 4
prediction: 3
THIS IS ROW 5

THIS IS ROW 6
choice: 0
THIS IS ROW 7
[CLS] children in england must ##n ' t work until they are 13 . they need to have a work permit to start working . the jobs teenagers can do delivering newspapers many teenagers will get up early to deliver newspapers to houses in their local area before going to school . they are known as paper - boys or paper ##girl ##s . baby ##sit ##ting : looking after young children in their home while their parents have gone out for the evening is a popular job for teenagers , as they get money for watching children and television all at the same time ! helping the milk ##man : from the age of 14 some teenagers help the milk ##man deliver milk to houses . other popular jobs : working in a shop ; office work ; washing cars ; in a cafe or restaurant . the hours teenagers ( 13 and 14 year olds ) can work : school days not more than 2 hours in one day dur

In [11]:
# count and calculate stats

import csv
from collections import defaultdict

skill_set = ["Lexical Matching",
            "Logical Reasoning",
            "Relation Identification",
            "Summary",
            "Coreference Resolution",
            "Algebraic Reasoning",
            "Commonsense Reasoning",
            "Special Questions"]

def calc_stats(csv_file, txt_file, count_by_cat=None, correct_by_cat=None,
              print_result = True):
    categorizations = csv.reader(open(csv_file), delimiter=",")

    if count_by_cat == None: count_by_cat = defaultdict(int)
    if correct_by_cat == None: correct_by_cat = defaultdict(int)

    def read_pred(file):
        for line in file:
            yield line

    pred_ite = read_pred(open(txt_file, "r"))
    curr_id = 0
    curr_question = 0
    header = True
    pqa = []
    for categorization in categorizations:
        if header:
            header = False
            continue
        target_id = int(categorization[0])
        target_question = int(categorization[1])
        skill_idx = int(categorization[2])
        target_answer = int(categorization[6])

        # loop until we find the target question
        while(True):
            pqa.clear()
            for i in range(17):
                pqa.append(next(pred_ite).strip()) # TODO:reinitialize
            if pqa[0][8:] != curr_id:
                curr_id = pqa[0][8:]
                curr_question = 1
            else:
                curr_question += 1
            if int(curr_id) == target_id and curr_question == target_question:
                break

        assert(target_answer == int(pqa[2][8]))

        count_by_cat[skill_idx] += 1
        if target_answer == int(pqa[3][12]):
            correct_by_cat[skill_idx] += 1

    if print_result:
        for skill_idx in range(8):
            print(skill_set[skill_idx])
            print("\tcount: ", count_by_cat[skill_idx])
            print("\tcorrect: ", correct_by_cat[skill_idx])
            if count_by_cat[skill_idx]==0: continue
            print("\taccuracy: ", correct_by_cat[skill_idx]/count_by_cat[skill_idx])
        
    return count_by_cat, correct_by_cat

In [12]:
cnt, correct = calc_stats("middle_categorization.csv", "BERT_Base/analyze_middle.txt", 
                          print_result=False)
calc_stats("high_categorization.csv", "BERT_Base/analyze_high.txt", cnt, correct)

Lexical Matching
	count:  150
	correct:  138
	accuracy:  0.92
Logical Reasoning
	count:  34
	correct:  18
	accuracy:  0.5294117647058824
Relation Identification
	count:  16
	correct:  3
	accuracy:  0.1875
Summary
	count:  42
	correct:  28
	accuracy:  0.6666666666666666
Coreference Resolution
	count:  7
	correct:  2
	accuracy:  0.2857142857142857
Algebraic Reasoning
	count:  8
	correct:  3
	accuracy:  0.375
Commonsense Reasoning
	count:  35
	correct:  13
	accuracy:  0.37142857142857144
Special Questions
	count:  7
	correct:  5
	accuracy:  0.7142857142857143


(defaultdict(int,
             {0: 150, 1: 34, 5: 8, 6: 35, 4: 7, 3: 42, 10: 1, 2: 16, 7: 7}),
 defaultdict(int,
             {0: 138, 1: 18, 4: 2, 5: 3, 3: 28, 10: 1, 2: 3, 6: 13, 7: 5}))

In [13]:
cnt, correct = calc_stats("middle_categorization.csv", "BERT_Large/analyze_middle.txt", 
                          print_result=False)
calc_stats("high_categorization.csv", "BERT_Large/analyze_high.txt", cnt, correct)

Lexical Matching
	count:  150
	correct:  132
	accuracy:  0.88
Logical Reasoning
	count:  34
	correct:  25
	accuracy:  0.7352941176470589
Relation Identification
	count:  16
	correct:  8
	accuracy:  0.5
Summary
	count:  42
	correct:  33
	accuracy:  0.7857142857142857
Coreference Resolution
	count:  7
	correct:  6
	accuracy:  0.8571428571428571
Algebraic Reasoning
	count:  8
	correct:  3
	accuracy:  0.375
Commonsense Reasoning
	count:  35
	correct:  17
	accuracy:  0.4857142857142857
Special Questions
	count:  7
	correct:  5
	accuracy:  0.7142857142857143


(defaultdict(int,
             {0: 150, 1: 34, 5: 8, 6: 35, 4: 7, 3: 42, 10: 1, 2: 16, 7: 7}),
 defaultdict(int,
             {0: 132, 1: 25, 4: 6, 6: 17, 3: 33, 10: 1, 2: 8, 5: 3, 7: 5}))

In [14]:
cnt, correct = calc_stats("middle_categorization.csv", "DCMN_Base/analyze_middle.txt", 
                          print_result=False)
calc_stats("high_categorization.csv", "DCMN_Base/analyze_high.txt", cnt, correct)

Lexical Matching
	count:  150
	correct:  135
	accuracy:  0.9
Logical Reasoning
	count:  34
	correct:  20
	accuracy:  0.5882352941176471
Relation Identification
	count:  16
	correct:  7
	accuracy:  0.4375
Summary
	count:  42
	correct:  28
	accuracy:  0.6666666666666666
Coreference Resolution
	count:  7
	correct:  4
	accuracy:  0.5714285714285714
Algebraic Reasoning
	count:  8
	correct:  1
	accuracy:  0.125
Commonsense Reasoning
	count:  35
	correct:  16
	accuracy:  0.45714285714285713
Special Questions
	count:  7
	correct:  5
	accuracy:  0.7142857142857143


(defaultdict(int,
             {0: 150, 1: 34, 5: 8, 6: 35, 4: 7, 3: 42, 10: 1, 2: 16, 7: 7}),
 defaultdict(int,
             {0: 135, 1: 20, 4: 4, 6: 16, 3: 28, 10: 1, 2: 7, 5: 1, 7: 5}))

In [5]:
calc_stats("middle_categorization.csv", "BERT_Base/analyze_middle.txt")

Lexical Matching
	count:  81
	correct:  73
	accuracy:  0.9012345679012346
Logical Reasoning
	count:  13
	correct:  8
	accuracy:  0.6153846153846154
Relation Identification
	count:  11
	correct:  2
	accuracy:  0.18181818181818182
Summary
	count:  14
	correct:  9
	accuracy:  0.6428571428571429
Coreference Resolution
	count:  5
	correct:  1
	accuracy:  0.2
Algebraic Reasoning
	count:  4
	correct:  2
	accuracy:  0.5
Commonsense Reasoning
	count:  20
	correct:  4
	accuracy:  0.2
Special Questions
	count:  1
	correct:  0
	accuracy:  0.0


In [6]:
calc_stats("high_categorization.csv", "BERT_Base/analyze_high.txt")

Lexical Matching
	count:  69
	correct:  65
	accuracy:  0.9420289855072463
Logical Reasoning
	count:  21
	correct:  10
	accuracy:  0.47619047619047616
Relation Identification
	count:  5
	correct:  1
	accuracy:  0.2
Summary
	count:  28
	correct:  19
	accuracy:  0.6785714285714286
Coreference Resolution
	count:  2
	correct:  1
	accuracy:  0.5
Algebraic Reasoning
	count:  4
	correct:  1
	accuracy:  0.25
Commonsense Reasoning
	count:  15
	correct:  9
	accuracy:  0.6
Special Questions
	count:  6
	correct:  5
	accuracy:  0.8333333333333334


In [7]:
calc_stats("middle_categorization.csv", "BERT_Large/analyze_middle.txt")

Lexical Matching
	count:  81
	correct:  75
	accuracy:  0.9259259259259259
Logical Reasoning
	count:  13
	correct:  10
	accuracy:  0.7692307692307693
Relation Identification
	count:  11
	correct:  5
	accuracy:  0.45454545454545453
Summary
	count:  14
	correct:  11
	accuracy:  0.7857142857142857
Coreference Resolution
	count:  5
	correct:  4
	accuracy:  0.8
Algebraic Reasoning
	count:  4
	correct:  1
	accuracy:  0.25
Commonsense Reasoning
	count:  20
	correct:  10
	accuracy:  0.5
Special Questions
	count:  1
	correct:  0
	accuracy:  0.0


In [8]:
calc_stats("high_categorization.csv", "BERT_Large/analyze_high.txt")

Lexical Matching
	count:  69
	correct:  57
	accuracy:  0.8260869565217391
Logical Reasoning
	count:  21
	correct:  15
	accuracy:  0.7142857142857143
Relation Identification
	count:  5
	correct:  3
	accuracy:  0.6
Summary
	count:  28
	correct:  22
	accuracy:  0.7857142857142857
Coreference Resolution
	count:  2
	correct:  2
	accuracy:  1.0
Algebraic Reasoning
	count:  4
	correct:  2
	accuracy:  0.5
Commonsense Reasoning
	count:  15
	correct:  7
	accuracy:  0.4666666666666667
Special Questions
	count:  6
	correct:  5
	accuracy:  0.8333333333333334


In [9]:
calc_stats("middle_categorization.csv", "DCMN_Base/analyze_middle.txt")

Lexical Matching
	count:  81
	correct:  75
	accuracy:  0.9259259259259259
Logical Reasoning
	count:  13
	correct:  9
	accuracy:  0.6923076923076923
Relation Identification
	count:  11
	correct:  5
	accuracy:  0.45454545454545453
Summary
	count:  14
	correct:  9
	accuracy:  0.6428571428571429
Coreference Resolution
	count:  5
	correct:  3
	accuracy:  0.6
Algebraic Reasoning
	count:  4
	correct:  1
	accuracy:  0.25
Commonsense Reasoning
	count:  20
	correct:  6
	accuracy:  0.3
Special Questions
	count:  1
	correct:  0
	accuracy:  0.0


In [10]:
calc_stats("high_categorization.csv", "DCMN_Base/analyze_high.txt")

Lexical Matching
	count:  69
	correct:  60
	accuracy:  0.8695652173913043
Logical Reasoning
	count:  21
	correct:  11
	accuracy:  0.5238095238095238
Relation Identification
	count:  5
	correct:  2
	accuracy:  0.4
Summary
	count:  28
	correct:  19
	accuracy:  0.6785714285714286
Coreference Resolution
	count:  2
	correct:  1
	accuracy:  0.5
Algebraic Reasoning
	count:  4
	correct:  0
	accuracy:  0.0
Commonsense Reasoning
	count:  15
	correct:  10
	accuracy:  0.6666666666666666
Special Questions
	count:  6
	correct:  5
	accuracy:  0.8333333333333334
